# Importing Packages and Inicial Data Preview

In [1]:
# Importing packages

import pandas as pd
import numpy as np

import requests
from pprint import pprint
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from category_encoders import LeaveOneOutEncoder

from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from itertools import combinations
from collections import Counter

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA

In [2]:
# loading dataset and handle a subset of it

df = pd.read_csv("US_Accidents_March23_sampled_500k.csv")

In [3]:
df.shape

(500000, 46)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 46 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     500000 non-null  object 
 1   Source                 500000 non-null  object 
 2   Severity               500000 non-null  int64  
 3   Start_Time             500000 non-null  object 
 4   End_Time               500000 non-null  object 
 5   Start_Lat              500000 non-null  float64
 6   Start_Lng              500000 non-null  float64
 7   End_Lat                279623 non-null  float64
 8   End_Lng                279623 non-null  float64
 9   Distance(mi)           500000 non-null  float64
 10  Description            499999 non-null  object 
 11  Street                 499309 non-null  object 
 12  City                   499981 non-null  object 
 13  County                 500000 non-null  object 
 14  State                  500000 non-nu

In [5]:
df.dtypes

ID                        object
Source                    object
Severity                   int64
Start_Time                object
End_Time                  object
Start_Lat                float64
Start_Lng                float64
End_Lat                  float64
End_Lng                  float64
Distance(mi)             float64
Description               object
Street                    object
City                      object
County                    object
State                     object
Zipcode                   object
Country                   object
Timezone                  object
Airport_Code              object
Weather_Timestamp         object
Temperature(F)           float64
Wind_Chill(F)            float64
Humidity(%)              float64
Pressure(in)             float64
Visibility(mi)           float64
Wind_Direction            object
Wind_Speed(mph)          float64
Precipitation(in)        float64
Weather_Condition         object
Amenity                     bool
Bump      

In [6]:
df.head(5)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-2047758,Source2,2,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,NaN,NaN,0.000,...,False,False,False,False,True,False,Day,Day,Day,Day
1,A-4694324,Source1,2,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,38.990037,-77.398282,0.056,...,False,False,False,False,False,False,Night,Night,Night,Night
2,A-5006183,Source1,2,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,34.661189,-120.492442,0.022,...,False,False,False,False,True,False,Day,Day,Day,Day
3,A-4237356,Source1,2,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,43.680574,-92.972223,1.054,...,False,False,False,False,False,False,Day,Day,Day,Day
4,A-6690583,Source1,2,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,35.395476,-118.985995,0.046,...,False,False,False,False,False,False,Night,Night,Night,Night


# Exploratory Data Analysis

### Data Overview

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Severity,500000.0,2.212748,0.486661,1.000000,2.000000,2.000000,2.000000,4.000000
Start_Lat,500000.0,36.206421,5.071411,24.562117,33.416823,35.832147,40.082443,48.999569
Start_Lng,500000.0,-94.736583,17.405761,-124.497420,-117.233047,-87.794365,-80.359601,-67.484130
End_Lat,279623.0,36.273192,5.265333,24.570110,33.474773,36.192669,40.181341,48.998901
End_Lng,279623.0,-95.776553,18.120211,-124.497419,-117.778324,-88.039013,-80.252449,-67.484130
Distance(mi),500000.0,0.564317,1.774872,0.000000,0.000000,0.029000,0.465000,193.479996
Temperature(F),489534.0,61.646254,19.000133,-77.800000,49.000000,64.000000,76.000000,207.000000
Wind_Chill(F),370983.0,58.229028,22.352246,-53.200000,43.000000,62.000000,75.000000,207.000000
Humidity(%),488870.0,64.834921,22.826158,1.000000,48.000000,67.000000,84.000000,100.000000
Pressure(in),491072.0,29.536621,1.008666,0.120000,29.370000,29.860000,30.030000,38.440000


In [8]:
df.isnull().sum()

ID                            0
Source                        0
Severity                      0
Start_Time                    0
End_Time                      0
Start_Lat                     0
Start_Lng                     0
End_Lat                  220377
End_Lng                  220377
Distance(mi)                  0
Description                   1
Street                      691
City                         19
County                        0
State                         0
Zipcode                     116
Country                       0
Timezone                    507
Airport_Code               1446
Weather_Timestamp          7674
Temperature(F)            10466
Wind_Chill(F)            129017
Humidity(%)               11130
Pressure(in)               8928
Visibility(mi)            11291
Wind_Direction            11197
Wind_Speed(mph)           36987
Precipitation(in)        142616
Weather_Condition         11101
Amenity                       0
Bump                          0
Crossing

In [9]:
missing_percentage = df.isna().sum().sort_values(ascending = False) / len(df) * 100
missing_percentage[missing_percentage !=0]

End_Lng                  44.0754
End_Lat                  44.0754
Precipitation(in)        28.5232
Wind_Chill(F)            25.8034
Wind_Speed(mph)           7.3974
Visibility(mi)            2.2582
Wind_Direction            2.2394
Humidity(%)               2.2260
Weather_Condition         2.2202
Temperature(F)            2.0932
Pressure(in)              1.7856
Weather_Timestamp         1.5348
Sunrise_Sunset            0.2966
Civil_Twilight            0.2966
Astronomical_Twilight     0.2966
Nautical_Twilight         0.2966
Airport_Code              0.2892
Street                    0.1382
Timezone                  0.1014
Zipcode                   0.0232
City                      0.0038
Description               0.0002
dtype: float64

In [10]:
df.duplicated().sum()

np.int64(0)

# Pre-processing

### Feature Selection/Extraction

In [11]:
# Extraction of the duration of accidents (witch is the same as time congestion)

df["Start_Time"] = pd.to_datetime(df["Start_Time"], format="mixed", errors='coerce', dayfirst=True)
df["End_Time"] = pd.to_datetime(df["End_Time"], format="mixed", errors='coerce', dayfirst=True)

df["Duration(min)"] = (df["End_Time"] - df["Start_Time"]).dt.total_seconds() / 60

# Checking the existence of outliers
print(len(df[df["Duration(min)"] < 0]))

0


In [12]:
# Extraction of year, month, weekday and day

df["Start_Time"] = pd.to_datetime(df["Start_Time"], format="mixed", errors='coerce', dayfirst=True)
df["Year"] = df["Start_Time"].dt.year
df["Month"] = df["Start_Time"].dt.month
df["Weekday"] = df["Start_Time"].dt.weekday
df["Day"] = df["Start_Time"].dt.day
df["Hour"] = df["Start_Time"].dt.hour

In [13]:
print("Number of unique Weather Conditions:", len(df["Weather_Condition"].unique()))
print("List of unique weather conditions:", list(df["Weather_Condition"].unique()))

Number of unique Weather Conditions: 109
List of unique weather conditions: ['Fair', 'Wintry Mix', 'Light Rain', 'Cloudy', 'Mostly Cloudy', 'Partly Cloudy', 'Clear', 'Scattered Clouds', 'Fog', 'Overcast', 'Light Snow', 'T-Storm', nan, 'Thunderstorms and Rain', 'Thunder', 'Light Rain with Thunder', 'Rain', 'Showers in the Vicinity', 'Mostly Cloudy / Windy', 'Heavy Rain', 'Cloudy / Windy', 'Light Drizzle', 'Heavy T-Storm', 'Light Rain / Windy', 'Smoke', 'Haze', 'Blowing Dust / Windy', 'N/A Precipitation', 'Thunder in the Vicinity', 'Snow', 'Heavy Thunderstorms and Rain', 'Shallow Fog', 'Light Freezing Drizzle', 'Fair / Windy', 'Patches of Fog', 'Light Snow / Windy', 'Blowing Snow / Windy', 'Thunderstorm', 'Drizzle', 'T-Storm / Windy', 'Partly Cloudy / Windy', 'Heavy Rain / Windy', 'Heavy Snow / Windy', 'Mist', 'Light Thunderstorms and Rain', 'Rain / Windy', 'Light Freezing Rain', 'Heavy Snow', 'Light Ice Pellets', 'Heavy T-Storm / Windy', 'Heavy Drizzle', 'Sleet', 'Light Rain Shower', 'H

In [14]:
# Reduce the number of weather conditions

# df.loc[df["Weather_Condition"].str.contains("Thunder|T-Storm", na=False), "Weather_Condition"] = "Thunderstorm"
df.loc[df["Weather_Condition"].str.contains("Snow|Sleet|Wintry", na=False), "Weather_Condition"] = "Snow"
df.loc[df["Weather_Condition"].str.contains("Rain|Drizzle|Shower", na=False), "Weather_Condition"] = "Rain"
df.loc[df["Weather_Condition"].str.contains("Wind|Squalls", na=False), "Weather_Condition"] = "Windy"
df.loc[df["Weather_Condition"].str.contains("Hail|Pellets", na=False), "Weather_Condition"] = "Hail"
df.loc[df["Weather_Condition"].str.contains("Fair", na=False), "Weather_Condition"] = "Clear"
df.loc[df["Weather_Condition"].str.contains("Cloud|Overcast", na=False), "Weather_Condition"] = "Cloudy"
df.loc[df["Weather_Condition"].str.contains("Mist|Haze|Fog", na=False), "Weather_Condition"] = "Fog"
df.loc[df["Weather_Condition"].str.contains("Sand|Dust", na=False), "Weather_Condition"] = "Sand"
df.loc[df["Weather_Condition"].str.contains("Smoke|Volcanic Ash", na=False), "Weather_Condition"] = "Smoke"
df.loc[df["Weather_Condition"].str.contains("N/A Precipitation", na=False), "Weather_Condition"] = "Unknown"
df['Weather_Condition'] = df['Weather_Condition'].fillna('Unknown')

list(df['Weather_Condition'].unique()) 

['Clear',
 'Snow',
 'Rain',
 'Cloudy',
 'Fog',
 'T-Storm',
 'Unknown',
 'Thunder',
 'Windy',
 'Heavy T-Storm',
 'Smoke',
 'Thunder in the Vicinity',
 'Thunderstorm',
 'Hail',
 'Sand',
 'Tornado']

In [15]:
# Bining of Severity (Congestion is not Severe or Severe)

df["Severity"] = df["Severity"].map(lambda x: 0 if x in [1, 2] else 1)

In [16]:
# Droping columns that are not relevant for the model

columns1 = ['ID','Source','End_Lat','End_Lng','End_Time','Start_Time','Description','Airport_Code','Country','Weather_Timestamp'
           ,'Nautical_Twilight','Astronomical_Twilight','Timezone','Wind_Direction','Zipcode','Wind_Chill(F)','Temperature(F)',
           'Sunrise_Sunset','Street','County','State','City','Precipitation(in)','Bump']

#  default
columns = ['ID','Source','End_Lat','End_Lng','End_Time','Start_Time','Description','Airport_Code','Country','Weather_Timestamp',
           'Civil_Twilight','Nautical_Twilight','Astronomical_Twilight','Timezone','Wind_Direction','Pressure(in)','Zipcode',
           'Precipitation(in)','Humidity(%)','Wind_Chill(F)','Temperature(F)','Sunrise_Sunset','Street','County',
           'State','City']
df1 = df.drop(columns=columns)

# MISSING VALUES

# Checking Missing Values

missing_vals = df1.isna().sum().sort_values(ascending = False) / len(df1) * 100
print(missing_vals[missing_vals !=0]) 

# =============================================================================
# ATTENTION! If you use columns1, there are more features with missing values:
# Features: Humidity(%), Pressure(in), Civil_Twilight
#
# Reduce Civil_Twilight to a binary variable
#
#df["Civil_Twilight"] = df["Civil_Twilight"].map(lambda x: 0 if x == "Night" else 1)
#
# Missing values
#
#df1.fillna({
#    'Humidity(%)': df['Humidity(%)'].median(),
#    'Pressure(in)': df['Pressure(in)'].median(),
#    'Civil_Twilight': df['Civil_Twilight'].mode()[0]},
#    inplace=True)
#
# =============================================================================

# Wind_Speed and Visibility Missing Values

df1.fillna({
    'Wind_Speed(mph)': df['Wind_Speed(mph)'].median(),
    'Visibility(mi)': df['Visibility(mi)'].median()},
    inplace=True)

# Checking once again the existence of Missing values

missing_vals = df1.isna().sum().sort_values(ascending = False) / len(df1) * 100
print(df1.shape)
print(df1.dtypes)
print(missing_vals[missing_vals !=0])

# PREPARING THE DATA BEFORE AND AFTER THE DATA SPLITTING

# Checking the class distribution before balancing
print("Before balancing:", Counter(df1['Severity']))

X = df1.drop(columns=['Severity'])
y = df1['Severity']

# Random Undersampling first to reduce dataset size

undersample = RandomUnderSampler(sampling_strategy=0.7, random_state=17)
X_resampled, y_resampled = undersample.fit_resample(X, y)

f1s_dt, precisions_dt, recalls_dt = [], [], []
f1s_knn, precisions_knn, recalls_knn = [], [], []
f1s_nb, precisions_nb, recalls_nb = [], [], []

n_runs = 5
for run in range(n_runs):

    # Spltting the data

    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=run)

    # LOO Encoding

    loo_encoder = LeaveOneOutEncoder()
    X_train_encoded = loo_encoder.fit_transform(X_train, y_train)
    X_test_encoded = loo_encoder.transform(X_test)

    # Apply Tomek Links to get better class separation
    # If you prefer not use tomek, consider changing y_train to y_tomek
    # in model training and changing X_train_encoded to X_tomek in standardization
    # And vice-versa!

    tomek = TomekLinks()
    X_tomek, y_tomek = tomek.fit_resample(X_train_encoded, y_train)  

    print("After Tomek Links:", Counter(y_tomek))

    # Doing Standardization after splitting to avoid data leakage

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_tomek)
    X_test_scaled = scaler.transform(X_test_encoded) 

    # Using PCA 

    pca = PCA(n_components=20) 

    X_train_scaled = pca.fit_transform(X_train_scaled)
    X_test_scaled = pca.transform(X_test_scaled)

    # Baseline models

    dt = DecisionTreeClassifier(max_depth=10)
    dt.fit(X_train_scaled, y_tomek)
    y_pred_dt = dt.predict(X_test_scaled)

    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train_scaled, y_tomek)
    y_pred_knn = knn.predict(X_test_scaled)

    nb = GaussianNB()
    nb.fit(X_train_scaled, y_tomek)
    y_pred_nb = nb.predict(X_test_scaled)

    precision_dt, recall_dt, f1_dt, _ = precision_recall_fscore_support(y_test, y_pred_dt, average='binary')
    f1s_dt.append(f1_dt)
    precisions_dt.append(precision_dt)
    recalls_dt.append(recall_dt)

    precision_knn, recall_knn, f1_knn, _ = precision_recall_fscore_support(y_test, y_pred_knn, average='binary')
    f1s_knn.append(f1_knn)
    precisions_knn.append(precision_knn)
    recalls_knn.append(recall_knn)

    precision_nb, recall_nb, f1_nb, _ = precision_recall_fscore_support(y_test, y_pred_nb, average='binary')
    f1s_nb.append(f1_nb)
    precisions_nb.append(precision_nb)
    recalls_nb.append(recall_nb)

results = {
        "Decision Tree": {"F1": round(np.mean(f1s_dt),2), "Precision": round(np.mean(precisions_dt),2), "Recall": round(np.mean(recalls_dt),2)},
        "KNN": {"F1": round(np.mean(f1s_knn),2), "Precision": round(np.mean(precisions_knn),2), "Recall": round(np.mean(recalls_knn),2)},
        "Naive Bayes": {"F1": round(np.mean(f1s_nb),2), "Precision": round(np.mean(precisions_nb),2), "Recall": round(np.mean(recalls_nb),2)}
    }
pprint(results)

Wind_Speed(mph)    7.3974
Visibility(mi)     2.2582
dtype: float64
(500000, 26)
Severity               int64
Start_Lat            float64
Start_Lng            float64
Distance(mi)         float64
Visibility(mi)       float64
Wind_Speed(mph)      float64
Weather_Condition     object
Amenity                 bool
Bump                    bool
Crossing                bool
Give_Way                bool
Junction                bool
No_Exit                 bool
Railway                 bool
Roundabout              bool
Station                 bool
Stop                    bool
Traffic_Calming         bool
Traffic_Signal          bool
Turning_Loop            bool
Duration(min)        float64
Year                   int32
Month                  int32
Weekday                int32
Day                    int32
Hour                   int32
dtype: object
Series([], dtype: float64)
Before balancing: Counter({0: 402416, 1: 97584})
After Tomek Links: Counter({0: 97548, 1: 78059})
After Tomek Links: Counter(